#### importing packages

In [1]:
import pandas as pd
# import matplotlib.pyplot as plt

from carriageway import Carriageway
from impact_factor import impact
from load import ll_A, ll_70R, ll_70RT
from reaction import find_bm, find_sf, find_ra, find_rb

# live load calculations

#### defining load (vehicles)
load pair contains pair (wheel position, wheel load)

In [2]:
vehicles = [ll_A, ll_70R, ll_70RT]
classA_pair, class70R, class70RT = [list(i.loadpair) for i in vehicles]
loads = [classA_pair, class70R, class70RT]

#### maxBM, maxSF at equal intervals
when each vehicle from the list of vehicles travels along the span, max reactions (BM and SF+/-) at equally divided intervals are calculated and stored in maxBMs, maxSFs_plus, maxSFs_minus.

In [3]:
span = 50

maxBMs = []
maxSFs_plus = []
maxSFs_minus = []
for i in range(len(loads)):
    maxBM = []
    maxSF_plus = []
    maxSF_minus = []
    for j in range(9):
        at = span / 8 * j
        first_wheel_at = 0
        step = 0.1

        BM = find_bm(span, 0, at)
        SF_plus = find_sf(span, 0, at)
        SF_minus = find_sf(span, 0, at)
        for k in range(int((span + loads[i][-1][0]) / step) + 2):  # '+2' to make sure the load moves all the way to
            # end until it has no effect
            bm = 0
            sf = 0
            for this in loads[i]:
                a, load = this
                pos = -a + first_wheel_at
                bm = bm + find_bm(span, pos, at) * load
                sf = sf + find_sf(span, pos, at) * load
            first_wheel_at += step
            BM = bm if bm > BM else BM
            SF_plus = sf if sf > SF_plus else SF_plus
            SF_minus = sf if sf < SF_minus else SF_minus
        maxBM.append(round(BM, 3))
        maxSF_plus.append(round(SF_plus, 3))
        maxSF_minus.append(round(SF_minus, 3))
    maxBMs.append(maxBM)
    maxSFs_plus.append(maxSF_plus)
    maxSFs_minus.append(maxSF_minus)

#### make a dataframe that contains maxBMs, maxSFs_plus, maxSFs_minus

In [4]:
A = ['MaxBM', 'MaxSF+', 'MaxSF-']
B = ['ClassA', 'Class70RW', 'Class70RT']

# A = ['ClassA', 'Class70RW', 'Class70RT']
# B = ['MaxBM', 'MaxSF+', 'MaxSF-']

iterables = [A, B]
index = pd.MultiIndex.from_product(iterables)

C = []

for i in [maxBMs, maxSFs_plus, maxSFs_minus]:
    C.extend(i)

# for i in range(len(loads)):
#     for j in [maxBMs, maxSFs_plus, maxSFs_minus]:
#         C.append(j[i])

df = pd.DataFrame(C, index=index, columns=[span / 8 * i for i in range(9)])
# print(df.loc[('ClassA', 'MaxSF-')])   ## you can navigate using loc, iloc

new_row = df.loc['MaxSF+'].where(df.loc['MaxSF+'] > abs(df.loc['MaxSF-']), abs(df.loc['MaxSF-']))

new_row.index = pd.MultiIndex.from_product([['MaxSF'], B])
df = pd.concat([df, new_row])

# df.to_excel('outputs/loads.xlsx') # uncomment to save as excel file

*how to read Excel file*

In [5]:
# df = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1])

# ###get index names###
# A = df.index.get_level_values(0).drop_duplicates().to_list()
# B = df.index.get_level_values(1).drop_duplicates().to_list()
# print(A, B)

#### impact factor

In [6]:
IF = [impact(i.name, span) for i in vehicles]
IF

[0.088, 0.088, 0.088]

#### possible combinations for given carriageway width

In [7]:
cwWidth = 6
carriageway = Carriageway(cwWidth)
combinations = carriageway.combinations()
combination_val = [i.get_value() for i in combinations]

#### eccentricity calculation
a combination consists of multiple arrangements gives different eccentricities. maximum eccentricities for each combination calculated.

In [8]:
list_combinations = []
list_max_e = []
list_plot = []
for i in combinations:
    list_combinations.append([i.classA, i.class70Rw, i.class70Rt])
    arrangements = i.arrangements()
    list_arrangements = []
    eccentricity = []
    max_e = 0
    for k in arrangements:
        if k.check_exceedance() * k.check_from_right() == 1:
            list_arrangements.append(k.veh)
            e = k.eccentricity()
            eccentricity.append(e)
            max_e = e if abs(e) > abs(max_e) else max_e
    list_max_e.append(max_e)

list_max_e

[-3.739698609263685e-16, -0.40500000000000025, -0.3500000000000001]

#### make a dataframe for combinations and their corresponding eccentricities

In [9]:
df1 = pd.DataFrame(list_combinations, columns=['ClassA', 'Class70Rw', 'Class70Rt'],
                  index=[f'comb{i + 1}' for i in range(len(list_combinations))])
df1['MaxEccentricity'] = list_max_e

*export eccentricity as excel file (uncomment)*

In [10]:
# df1.to_excel('outputs/max_e.xlsx')

#### total reactions for given combination
total reactions = sum of (maxreactions at mid span * impact factor * no. of that vehicles)

In [11]:
totalBMs = [(df.loc['MaxBM'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
totalSFs = [(df.loc['MaxSF'][span/2]*IF).dot(combination_val[i]) for i in range(len(combinations))]
combination_val, totalBMs, totalSFs

([[2, 0, 0], [0, 1, 0], [0, 0, 1]],
 [972.6023999999999, 957.2112, 734.1003999999999],
 [30.828512, 34.982112, 27.947919999999996])

In [12]:
df

0.00      6.25      12.50      18.75      25.00  \
MaxBM  ClassA       0.000  2361.037  4068.075   5120.737   5526.150   
       Class70RW    0.000  4826.025  8200.950  10182.475  10877.400   
       Class70RT    0.000  3651.438  6259.675   7823.212   8342.050   
MaxSF+ ClassA     445.318   376.622   307.926    238.122    169.426   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   
MaxSF- ClassA       0.000   -12.030   -51.234   -106.602   -175.162   
       Class70RW    0.000   -17.024   -87.536   -208.476   -332.476   
       Class70RT    0.000   -54.810  -143.010   -229.810   -316.610   
MaxSF  ClassA     445.318   376.622   307.926    238.122    175.162   
       Class70RW  895.524   771.524   647.524    521.524    397.524   
       Class70RT  667.590   579.390   492.590    404.390    317.590   

                      31.25     37.50     43.75    50.00  
MaxBM  ClassA      5340.038  4327.675  2494.638    0.000  
       Class70RW  10163.225  8053.750  4611.975    0.000  
       Class70RT   7823.588  6260.425  3652.563    0.000  
MaxSF+ ClassA        99.676    43.520    13.056   -0.000  
       Class70RW    271.524   148.964    44.948   -0.000  
       Class70RT    229.390   142.590    54.390   -0.000  
MaxSF- ClassA      -244.966  -313.662  -383.466 -452.162  
       Class70RW   -458.476  -582.476  -708.476 -832.476  
       Class70RT   -404.810  -491.610  -579.810 -666.610  
MaxSF  ClassA       244.966   313.662   383.466  452.162  
       Class70RW    458.476   582.476   708.476  832.476  
       Class70RT    404.810   491.610   579.810  666.610

In [13]:
df1['TotalBM'] = totalBMs
df1['TotalSF'] = totalSFs

In [14]:
df1

,ClassA,Class70Rw,Class70Rt,MaxEccentricity,TotalBM,TotalSF
comb1,2,0,0,-3.739699e-16,972.6024,30.828512
comb2,0,1,0,-4.050000e-01,957.2112,34.982112
comb3,0,0,1,-3.500000e-01,734.1004,27.947920


# loads on peir

In [15]:
import math

In [16]:
abs(math.log10(step))

1.0

In [32]:
span = 30
c_c = 1.5
reactions = []

max_sums = []
lefts = []
rights = []
max_sums_at = []
for veh in vehicles:
    dummy_load = list(veh.loadpair)
    no = 2
    nose_dist = 20
    if veh.name == 'Class 70RT':
        nose_dist = 90
        no = 1
    elif veh.name == 'Class 70RW':
        nose_dist = 31.52
        
    final_load = list(dummy_load)
    max_sum_at = 0
    for i in range(no-1):
        delta = dummy_load[-1][0] + nose_dist
        final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
        
    sumlr = 0
    nleft = 0
    nright = 0
    
    head_at = 0
    step = 0.01
    for i in range(int(90/step)):
        left = 0
        right = 0
        for j in final_load:
            rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*j[1]
            ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*j[1]
            temp_load = 0
            if head_at-j[0] == span+c_c/2:
#                 print('hjgjh')
                temp_load = (j[1])/2
                ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*temp_load
                rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*temp_load
            left+=rb
            right+=ra
        
        if left+right>sumlr:
            sumlr = left+right
            nleft, nright = left, right
            max_sum_at = head_at
            
        head_at= round(head_at + step, int(abs(math.log10(step))))
    max_sums.append(sumlr)
    lefts.append(nleft)
    rights.append(nright)
    max_sums_at.append(max_sum_at)

In [33]:
pd.DataFrame([lefts, rights, max_sums, max_sums_at], columns=['class A', '70RW', '70RT'], index=['RL', "RR", 'sum', 'max_load_at'])

,class A,70RW,70RT
RL,199.104000,480.42,341.826667
RR,292.440667,436.46,348.476667
sum,491.544667,916.88,690.303333
max_load_at,75.060000,39.73,32.750000


In [19]:
find_ra(30, -0.75, 0.75, 0.75)

1.025

*check*

In [24]:
span = 30
c_c = 1.5
veh = ll_A
dummy_load = list(veh.loadpair)
no = 2
nose_dist = 20
if veh.name == 'Class 70RT':
    nose_dist = 90
    no = 1
elif veh.name == 'Class 70RW':
    nose_dist = 31.52

final_load = list(dummy_load)
for i in range(no-1):
    delta = dummy_load[-1][0] + nose_dist
    final_load.extend([(i[0] + delta, i[1]) for i in dummy_load])
    
left = []
right = []
head_at = 75.05

for j in final_load:
    rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*j[1]
    ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*j[1]
    temp_load = 0
    if head_at-j[0] == span+c_c/2:
#         print('hjgjh')
        temp_load = (j[1])/2
        ra = find_ra(span, head_at-j[0]-(span+c_c), c_c/2, c_c/2)*temp_load
        rb = find_rb(span, head_at-j[0], c_c/2, c_c/2)*temp_load
    left.append(rb)
    right.append(ra)

In [25]:
pd.DataFrame({'pos' : [head_at - i[0] for i in final_load], 'load' : [i[1] for i in final_load], 'left' : left, 'right' : right} )

,pos,load,left,right
0,75.05,27,0.000000,0.000
1,73.95,27,0.000000,0.000
2,70.75,114,0.000000,0.000
3,69.55,114,0.000000,0.000
4,65.25,68,0.000000,0.000
5,62.25,68,0.000000,-1.700
6,59.25,68,0.000000,5.100
7,56.25,68,0.000000,11.900
8,36.25,27,0.000000,22.725
9,35.15,27,0.000000,23.715


In [22]:
sum(left), sum(right), sum(left)+sum(right)

(315.86333333333334, 290.88, 606.7433333333333)